In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import transform
import matplotlib.pyplot as plt
from model import get_vgg_model

In [2]:
model = get_vgg_model()

In [3]:
img = plt.imread("images/style/wave.jpg")

In [4]:
def parse_image(img, size=(224, 224)):
    return tf.image.resize(img, size)
def process_image_for_vgg_model(img):
    processed = tf.keras.applications.vgg19.preprocess_input(img)
    return parse_image(processed)

In [13]:
extractor = tf.keras.Model(inputs=model.inputs, outputs=[model.layers[21].output])

In [14]:
res = extractor(np.array([process_image_for_vgg_model(img)]))

In [25]:
res.get_shape()

TensorShape([1, 28, 28, 512])

In [27]:
tf.reshape(res, (28, 28 * 512))

<tf.Tensor: shape=(28, 14336), dtype=float32, numpy=
array([[   0.      ,    0.      ,   29.23805 , ...,  105.466034,
         112.91182 ,    0.      ],
       [   0.      ,    0.      ,    0.      , ...,    0.      ,
         353.00723 ,    0.      ],
       [ 471.05264 ,    0.      ,    0.      , ...,    0.      ,
         328.0361  ,    0.      ],
       ...,
       [2485.288   ,    0.      ,    0.      , ..., 3062.9434  ,
        1515.7535  , 2407.9702  ],
       [1523.0756  ,    0.      ,    0.      , ..., 1672.4797  ,
        2014.6669  , 1437.8273  ],
       [ 539.79083 ,  368.05673 ,    0.      , ...,  945.0028  ,
         981.1016  ,  943.4508  ]], dtype=float32)>